In [ ]:
import os
import numpy as np
import rasterio
from geopy.distance import geodesic
from tqdm import tqdm

# Function to calculate lookup table for pixel widths
def compute_width_lookup(num_rows, transform):
    latitudes = [transform[5] + i * transform[4] for i in range(num_rows)]
    return [compute_width(lat, abs(transform[0])) for lat in latitudes]

# Function to calculate pixel width
def compute_width(lat, cellsize):
    point1 = (lat, 0)
    point2 = (lat, cellsize)
    return geodesic(point1, point2).meters

# Function to calculate pixel height
def compute_height(cellsize):
    point1 = (0, 0)
    point2 = (0, cellsize)
    return geodesic(point1, point2).meters

# Function to compute forest edge length
def compute_edge_length(i, j, forest, width_lookup, height):
    edge_length = 0.0  # Initialize edge length

    # Check left pixel
    if j > 0 and not forest[i, j-1]:
        edge_length += height  # Add height to edge length

    # Check right pixel
    if j < forest.shape[1] - 1 and not forest[i, j+1]:
        edge_length += height  # Add height to edge length

    # Check upper pixel
    if i > 0 and not forest[i-1, j]:
        edge_length += width_lookup[i-1]  # Add width of upper pixel to edge length

    # Check lower pixel
    if i < forest.shape[0] - 1 and not forest[i+1, j]:
        edge_length += width_lookup[i]  # Add width of current pixel to edge length

    return edge_length

# Paths to input and output folders
input_folder = "H:\\Global_tree_cover\\2000extent"
output_folder_area = "H:\\Global_tree_cover\\2000Area"
output_folder_edge = "H:\\Global_tree_cover\\2000Edge"

# Create output folders if they don't exist
if not os.path.exists(output_folder_area):
    os.makedirs(output_folder_area)
if not os.path.exists(output_folder_edge):
    os.makedirs(output_folder_edge)

# Get list of tif files
tif_files = [f for f in os.listdir(input_folder) if f.endswith(".tif")]

# Loop through each tif file in the input folder
for i, filename in enumerate(tqdm(tif_files, desc="Processing files")):
    tqdm.write(f"Processing file {i+1}/{len(tif_files)}: {filename}")

    # Full paths to input and output files
    input_file = os.path.join(input_folder, filename)
    output_file_area = os.path.join(output_folder_area, filename)
    output_file_edge = os.path.join(output_folder_edge, filename)

    # Open tif file
    with rasterio.open(input_file) as ds:
        forest = ds.read(1).astype(bool)
        transform = ds.transform
        cellsize = abs(transform[0])

        # Compute lookup table for pixel widths and pixel height
        width_lookup = compute_width_lookup(forest.shape[0], transform)
        height = compute_height(cellsize)

        # Create new tifs
        area_tif = np.zeros_like(forest, dtype=np.float32)
        edge_tif = np.zeros_like(forest, dtype=np.float32)

        # Calculate the total number of pixels in the TIFF file
        total_pixels = forest.size
        
        # Determine the update interval for the nested tqdm progress bar
        update_interval = max(1, total_pixels // 10)  # Adjust the denominator to control the frequency of updates

        # Initialize the nested tqdm progress bar for each TIFF file
        with tqdm(total=total_pixels, desc="Processing", leave=False) as pbar:
            # Loop through each pixel in the forest tif
            for i in range(forest.shape[0]):
                for j in range(forest.shape[1]):
                    # Check if the pixel is a forest pixel
                    if forest[i, j]:
                        # Calculate the area of the pixel and assign it to area_tif
                        area_tif[i, j] = width_lookup[i] * height

                        # Calculate the edge length of the pixel and assign it to edge_tif
                        edge_tif[i, j] = compute_edge_length(i, j, forest, width_lookup, height)

                    # Update the nested tqdm progress bar at the specified interval
                    if (i * forest.shape[1] + j) % update_interval == 0:
                        pbar.update(update_interval)

        # Write new tif to file
        with rasterio.open(output_file_area, 'w', driver='GTiff', height=area_tif.shape[0],
                           width=area_tif.shape[1], count=1, dtype=area_tif.dtype,
                           crs=ds.crs, transform=transform) as dst:
            dst.write(area_tif, 1)

        with rasterio.open(output_file_edge, 'w', driver='GTiff', height=edge_tif.shape[0],
                           width=edge_tif.shape[1], count=1, dtype=edge_tif.dtype,
                           crs=ds.crs, transform=transform) as dst:
            dst.write(edge_tif, 1)

Processing files:   0%|                                                                        | 0/261 [00:00<?, ?it/s]

Processing file 1/261: 30N_110E.tif



Processing:   7%|███▎                                               | 105600000/1600000000 [02:37<39:51, 624998.50it/s]


Processing:  13%|██████▋                                            | 211200000/1600000000 [05:11<38:27, 601844.49it/s]


Processing:  20%|██████████                                         | 316800000/1600000000 [08:05<35:14, 606748.45it/s]


...


Processing:   7%|███▎                                              | 105600000/1600000000 [00:51<12:28, 1996857.81it/s]


Processing:  13%|██████▌                                           | 211200000/1600000000 [01:45<11:56, 1938970.81it/s]


Processing:  20%|█████████▉                                        | 316800000/1600000000 [02:37<10:32, 2029472.39it/s]


Processing:  26%|█████████████▏                                    | 422400000/1600000000 [03:30<09:46, 2007619.61it/s]


Processing:  29%|██████████████▋                                   | 468800000/1600000000 [03:53<09:18, 2026780.99it/s]

In [3]:
import os
import numpy as np
import rasterio
from geopy.distance import geodesic
from tqdm import tqdm
import gc

# Function to calculate lookup table for pixel widths
def compute_width_lookup(num_rows, transform):
    latitudes = [transform[5] + i * transform[4] for i in range(num_rows)]
    return [compute_width(lat, abs(transform[0])) for lat in latitudes]

# Function to calculate pixel width
def compute_width(lat, cellsize):
    point1 = (lat, 0)
    point2 = (lat, cellsize)
    return geodesic(point1, point2).meters

# Function to calculate pixel height
def compute_height(cellsize):
    point1 = (0, 0)
    point2 = (0, cellsize)
    return geodesic(point1, point2).meters

# Function to compute forest edge length
def compute_edge_length(i, j, forest, width_lookup, height):
    edge_length = 0.0  # Initialize edge length

    # Check left pixel
    if j > 0 and not forest[i, j-1]:
        edge_length += height  # Add height to edge length

    # Check right pixel
    if j < forest.shape[1] - 1 and not forest[i, j+1]:
        edge_length += height  # Add height to edge length

    # Check upper pixel
    if i > 0 and not forest[i-1, j]:
        edge_length += width_lookup[i-1]  # Add width of upper pixel to edge length

    # Check lower pixel
    if i < forest.shape[0] - 1 and not forest[i+1, j]:
        edge_length += width_lookup[i]  # Add width of current pixel to edge length

    return edge_length

# Paths to input and output folders
input_folder = "H:\\Global_tree_cover\\2000extent"
output_folder_area = "H:\\Global_tree_cover\\TIFoutput\\2000Area"
output_folder_edge = "H:\\Global_tree_cover\\TIFoutput\\2000Edge"

# Create output folders if they don't exist
if not os.path.exists(output_folder_area):
    os.makedirs(output_folder_area)
if not os.path.exists(output_folder_edge):
    os.makedirs(output_folder_edge)

# Get list of tif files
tif_files = [f for f in os.listdir(input_folder) if f.endswith(".tif")]

# Loop through each tif file in the input folder
for i, filename in enumerate(tqdm(tif_files, desc="Processing files")):
    tqdm.write(f"Processing file {i+1}/{len(tif_files)}: {filename}")

    # Full paths to input and output files
    input_file = os.path.join(input_folder, filename)
    output_file_area = os.path.join(output_folder_area, filename)
    output_file_edge = os.path.join(output_folder_edge, filename)

    # Open tif file
    with rasterio.open(input_file) as ds:
        forest = ds.read(1).astype(bool)
        transform = ds.transform
        cellsize = abs(transform[0])

        # Compute lookup table for pixel widths and pixel height
        width_lookup = compute_width_lookup(forest.shape[0], transform)
        height = compute_height(cellsize)

        # Create new tifs
        area_tif = np.zeros_like(forest, dtype=np.float32)
        edge_tif = np.zeros_like(forest, dtype=np.float32)

        # Calculate the total number of pixels in the TIFF file
        total_pixels = forest.size
        

        # Loop through each pixel in the forest tif
        for i in range(forest.shape[0]):
            for j in range(forest.shape[1]):
                # Check if the pixel is a forest pixel
                if forest[i, j]:
                        # Calculate the area of the pixel and assign it to area_tif
                        area_tif[i, j] = width_lookup[i] * height

                        # Calculate the edge length of the pixel and assign it to edge_tif
                        edge_tif[i, j] = compute_edge_length(i, j, forest, width_lookup, height)

        # Write new tif to file
        with rasterio.open(output_file_area, 'w', driver='GTiff', height=area_tif.shape[0],
                           width=area_tif.shape[1], count=1, dtype=area_tif.dtype,
                           crs=ds.crs, transform=transform) as dst:
            dst.write(area_tif, 1)

        with rasterio.open(output_file_edge, 'w', driver='GTiff', height=edge_tif.shape[0],
                           width=edge_tif.shape[1], count=1, dtype=edge_tif.dtype,
                           crs=ds.crs, transform=transform) as dst:
            dst.write(edge_tif, 1)
        del forest
        del area_tif
        del edge_tif
        gc.collect()

Processing files:   0%|                                                                        | 0/261 [00:00<?, ?it/s]

Processing file 1/261: 30N_110E.tif


Processing files:   0%|▏                                                         | 1/261 [28:58<125:33:33, 1738.51s/it]

Processing file 2/261: 30N_110W.tif


Processing files:   1%|▍                                                          | 2/261 [44:13<90:13:47, 1254.16s/it]

...

Processing file 74/261: 50S_080W.tif


Processing files:  28%|███████████████▉                                        | 74/261 [16:03:37<37:43:49, 726.36s/it]

Processing file 75/261: 60N_000E.tif


Processing files:  28%|███████████████▉                                        | 74/261 [16:13:22<40:59:45, 789.23s/it]


RasterioIOError: H:/Global_tree_cover/TIFoutput/2000Area/60N_000E.tif: Free disk space available is 3383230464 bytes, whereas 6400000000 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.